In [1]:
import numpy as np
import matplotlib.pyplot as plt

import stan_utils as stan
from mpl_utils import (mpl_style, common_limits)

plt.style.use(mpl_style)

np.random.seed(42)

%matplotlib inline

In [13]:
def generate_data(N, M, D, overlap_fraction=None, scales=None,
                  phi_scale=1, seed=None):
    """
    Generate some toy data to play with. Here we assume all :math:`N` stars have
    been observed by all :math:`M` surveys.

    :param N:
        The number of stars observed.

    :param M:
        The number of surveys.

    :param D:
        The dimensionality of the label space.

    :param overlap_fraction: [optional]
        The approximate fraction of stars observed by each survey. If `None` is
        provided then some random ~50% of stars will be observed by each survey.
        If provided, this should be a list of length `M` with values between 0
        (no overlap) and 1 (complete overlap).

    :param scales: [optional]
        Optional values to provide for the relative scales on the latent factors.

    :param seed: [optional]
        An optional seed to provide to the random number generator.
        
    :param phi_scale: [optional]
        An optional scale value for the uncertainties.

    :returns:
        A two-length tuple containing the data :math:`y` and a dictionary with
        the true values.
    """

    if seed is not None:
        np.random.seed(seed)

    if scales is None:
        scales = np.abs(np.random.normal(0, 1, size=D))

    else:
        scales = np.array(scales)
        
    if overlap_fraction is None:
        overlap_fraction = np.random.normal(0.5, 0.05, size=M)
    else:
        overlap_fraction = np.array(overlap_fraction)
        assert  overlap_fraction.size == M
        

    assert len(scales) == D

    X = np.random.normal(
        np.zeros(D),
        scales,
        size=(N, D))

    # TODO: Better way to randomly generate positive semi-definite covariance
    #       matrices that are *very* close to an identity matrix.

    # Use decomposition to ensure the resulting covariance matrix is positive
    # semi-definite.
    L = np.random.randn(M, D, D)
    L[:, np.arange(D), np.arange(D)] = np.exp(L[:, np.arange(D), np.arange(D)])
    i, j = np.triu_indices_from(L[0], 1)
    L[:, i, j] = 0.0

    # TODO: use matrix multiplication you idiot
    theta = np.array([np.dot(L[i], L[i].T) for i in range(M)])

    y = np.dot(X, theta)
    for m, of in enumerate(overlap_fraction):
        indices = np.where(np.random.uniform(0, 1, size=N) <= of)[0]
        y[indices, m, :] = np.nan
        
    # add noise.
    phi = np.abs(np.random.normal(0, phi_scale, size=(M, D)))
    rank = np.random.normal(0, 1, size=y.shape)

    noise = scales * rank * phi 

    y += noise

    truths = dict(X=X, theta=theta, phi=phi, scales=scales, L=L, noise=noise)

    return (y, truths)

In [14]:
N, M, D = (250, 10, 5)
scales = np.ones(D)
phi_scale = 1

y, truths = generate_data(N=N, M=M, D=D, 
                          scales=scales, phi_scale=phi_scale)

In [27]:
# 

10